# Aggregating counts and rates of hospital admissions and make a table

In [663]:
import pandas as pd
import numpy as np
import functools as ft
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

## 0- functions and read data

In [664]:
def data_process(data, data_hosped, infection):
    #replace SMs with 5 in data and data_hosped and NaN with 0 in data_hosped 
    data = data.replace({'SM': 5})
    data_hosped = data_hosped.replace({'SM': 5, np.nan:0})
    #conver counts into int
    data[infection] = data[infection].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped[infection] = data_hosped[infection].astype(str).replace('\.0', '', regex=True).astype(int)

    #add hosp rate
    data_hosped[infection+'_rate'] = (data_hosped[infection]/data[infection])*1000
    data_hosped[infection+'_rate'] = np.round(data_hosped[infection+'_rate'], decimals=1)

    #replace rate with SN if infection count equals 5
    data_hosped.loc[data_hosped[infection] == 5, infection+'_rate'] = 'SN'

    #calculate total using sex_female and sex_male
    total = data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0])][infection] + data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0])][infection]
    total = "{:,}".format(total)
    data_hosped['variable'] = data_hosped['variable'].str.replace('antibacterial_brit', 'total')
    
    #add comma to thousands of column count
    data_hosped[infection] = data_hosped[infection].apply(lambda x : "{:,}".format(x))
    #make new column with counts and rates
    data_hosped[infection+'_rate_count'] = data_hosped[infection].astype(str) + ' (' + data_hosped[infection+'_rate'].astype(str) + ')'
    
    #replace total (after making new column of counts and rates)
    data_hosped.at[int(data_hosped[data_hosped['variable']=='total'].index[0]),infection+'_rate_count'] = total

    #drop redundant columns and rename the needed column
    data_hosped = data_hosped.drop([infection, infection+'_rate'], axis=1)
    data_hosped = data_hosped.rename(columns={infection+'_rate_count': infection})
    #replace any row with no value (=0) with a dash 
    data_hosped[infection] = data_hosped[infection].replace({'0 (0.0)': '-'})
    #replace count and rate if rate contains SN
    data_hosped.loc[data_hosped[infection].str.contains('SN', regex=False), infection] = 'SN' 

    return data_hosped

In [665]:
summary_table_hosped_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt_hosped.csv')
summary_table_hosped_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt_hosped.csv')
summary_table_hosped_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl_hosped.csv')
summary_table_hosped_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl_hosped.csv')
summary_table_hosped_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt_hosped.csv')
summary_table_hosped_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt_hosped.csv')
summary_table_hosped_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl_hosped.csv')
summary_table_hosped_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl_hosped.csv')
summary_table_hosped_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt_hosped.csv')
summary_table_hosped_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt_hosped.csv')
summary_table_hosped_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl_hosped.csv')
summary_table_hosped_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl_hosped.csv')
summary_table_hosped_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt_hosped.csv')
summary_table_hosped_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt_hosped.csv')
summary_table_hosped_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl_hosped.csv')
summary_table_hosped_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl_hosped.csv')
summary_table_hosped_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt_hosped.csv')
summary_table_hosped_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt_hosped.csv')
summary_table_hosped_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl_hosped.csv')
summary_table_hosped_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl_hosped.csv')
summary_table_hosped_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt_hosped.csv')
summary_table_hosped_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt_hosped.csv')
summary_table_hosped_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl_hosped.csv')
summary_table_hosped_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl_hosped.csv')
summary_table_hosped_pneumonia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_incdt_hosped.csv')
summary_table_hosped_pneumonia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_incdt_hosped.csv')
summary_table_hosped_pneumonia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_prevl_hosped.csv')
summary_table_hosped_pneumonia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_prevl_hosped.csv')
summary_table_hosped_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt_hosped.csv')
summary_table_hosped_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt_hosped.csv')
summary_table_hosped_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl_hosped.csv')
summary_table_hosped_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl_hosped.csv')
summary_table_hosped_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt_hosped.csv')
summary_table_hosped_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt_hosped.csv')
summary_table_hosped_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl_hosped.csv')
summary_table_hosped_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl_hosped.csv')
summary_table_hosped_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt_hosped.csv')
summary_table_hosped_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt_hosped.csv')
summary_table_hosped_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl_hosped.csv')
summary_table_hosped_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl_hosped.csv')
summary_table_hosped_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt_hosped.csv')
summary_table_hosped_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt_hosped.csv')
summary_table_hosped_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl_hosped.csv')
summary_table_hosped_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl_hosped.csv')

In [666]:
summary_table_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt.csv')
summary_table_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt.csv')
summary_table_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl.csv')
summary_table_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl.csv')
summary_table_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt.csv')
summary_table_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt.csv')
summary_table_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl.csv')
summary_table_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl.csv')
summary_table_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt.csv')
summary_table_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt.csv')
summary_table_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl.csv')
summary_table_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl.csv')
summary_table_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt.csv')
summary_table_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt.csv')
summary_table_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl.csv')
summary_table_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl.csv')
summary_table_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt.csv')
summary_table_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt.csv')
summary_table_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl.csv')
summary_table_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl.csv')
summary_table_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt.csv')
summary_table_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt.csv')
summary_table_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl.csv')
summary_table_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl.csv')
summary_table_pneumonia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_incdt.csv')
summary_table_pneumonia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_incdt.csv')
summary_table_pneumonia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_no_abs_prevl.csv')
summary_table_pneumonia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_pneumonia/summary_table_pneumonia_abs_prevl.csv')
summary_table_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt.csv')
summary_table_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt.csv')
summary_table_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl.csv')
summary_table_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl.csv')
summary_table_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt.csv')
summary_table_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt.csv')
summary_table_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl.csv')
summary_table_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl.csv')
summary_table_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt.csv')
summary_table_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt.csv')
summary_table_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl.csv')
summary_table_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl.csv')
summary_table_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt.csv')
summary_table_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt.csv')
summary_table_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl.csv')
summary_table_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl.csv')

In [667]:
#select relevant columns of each df
summary_table_lrti_no_abs_incdt = summary_table_lrti_no_abs_incdt[['variable', 'count']]
summary_table_lrti_abs_incdt = summary_table_lrti_abs_incdt[['variable', 'count']]
summary_table_lrti_no_abs_prevl = summary_table_lrti_no_abs_prevl[['variable', 'count']]
summary_table_lrti_abs_prevl = summary_table_lrti_abs_prevl[['variable', 'count']]
summary_table_urti_combined_no_abs_incdt = summary_table_urti_combined_no_abs_incdt[['variable', 'count']]
summary_table_urti_combined_abs_incdt = summary_table_urti_combined_abs_incdt[['variable', 'count']]
summary_table_urti_combined_no_abs_prevl = summary_table_urti_combined_no_abs_prevl[['variable', 'count']]
summary_table_urti_combined_abs_prevl = summary_table_urti_combined_abs_prevl[['variable', 'count']]
summary_table_uti_no_abs_incdt = summary_table_uti_no_abs_incdt[['variable', 'count']]
summary_table_uti_abs_incdt = summary_table_uti_abs_incdt[['variable', 'count']]
summary_table_uti_no_abs_prevl = summary_table_uti_no_abs_prevl[['variable', 'count']]
summary_table_uti_abs_prevl = summary_table_uti_abs_prevl[['variable', 'count']]
summary_table_pneumonia_no_abs_incdt = summary_table_pneumonia_no_abs_incdt[['variable', 'count']]
summary_table_pneumonia_abs_incdt = summary_table_pneumonia_abs_incdt[['variable', 'count']]
summary_table_pneumonia_no_abs_prevl = summary_table_pneumonia_no_abs_prevl[['variable', 'count']]
summary_table_pneumonia_abs_prevl = summary_table_pneumonia_abs_prevl[['variable', 'count']]
summary_table_sinusitis_no_abs_incdt = summary_table_sinusitis_no_abs_incdt[['variable', 'count']]
summary_table_sinusitis_abs_incdt = summary_table_sinusitis_abs_incdt[['variable', 'count']]
summary_table_sinusitis_no_abs_prevl = summary_table_sinusitis_no_abs_prevl[['variable', 'count']]
summary_table_sinusitis_abs_prevl = summary_table_sinusitis_abs_prevl[['variable', 'count']]
summary_table_otmedia_no_abs_incdt = summary_table_otmedia_no_abs_incdt[['variable', 'count']]
summary_table_otmedia_abs_incdt = summary_table_otmedia_abs_incdt[['variable', 'count']]
summary_table_otmedia_no_abs_prevl = summary_table_otmedia_no_abs_prevl[['variable', 'count']]
summary_table_otmedia_abs_prevl = summary_table_otmedia_abs_prevl[['variable', 'count']]
summary_table_ot_externa_no_abs_incdt = summary_table_ot_externa_no_abs_incdt[['variable', 'count']]
summary_table_ot_externa_abs_incdt = summary_table_ot_externa_abs_incdt[['variable', 'count']]
summary_table_ot_externa_no_abs_prevl = summary_table_ot_externa_no_abs_prevl[['variable', 'count']]
summary_table_ot_externa_abs_prevl = summary_table_ot_externa_abs_prevl[['variable', 'count']]
summary_table_urti_no_abs_incdt = summary_table_urti_no_abs_incdt[['variable', 'count']]
summary_table_urti_abs_incdt = summary_table_urti_abs_incdt[['variable', 'count']]
summary_table_urti_no_abs_prevl = summary_table_urti_no_abs_prevl[['variable', 'count']]
summary_table_urti_abs_prevl = summary_table_urti_abs_prevl[['variable', 'count']]
summary_table_cough_no_abs_incdt = summary_table_cough_no_abs_incdt[['variable', 'count']]
summary_table_cough_abs_incdt = summary_table_cough_abs_incdt[['variable', 'count']]
summary_table_cough_no_abs_prevl = summary_table_cough_no_abs_prevl[['variable', 'count']]
summary_table_cough_abs_prevl = summary_table_cough_abs_prevl[['variable', 'count']]
summary_table_cough_cold_no_abs_incdt = summary_table_cough_cold_no_abs_incdt[['variable', 'count']]
summary_table_cough_cold_abs_incdt = summary_table_cough_cold_abs_incdt[['variable', 'count']]
summary_table_cough_cold_no_abs_prevl = summary_table_cough_cold_no_abs_prevl[['variable', 'count']]
summary_table_cough_cold_abs_prevl = summary_table_cough_cold_abs_prevl[['variable', 'count']]
summary_table_throat_no_abs_incdt = summary_table_throat_no_abs_incdt[['variable', 'count']]
summary_table_throat_abs_incdt = summary_table_throat_abs_incdt[['variable', 'count']]
summary_table_throat_no_abs_prevl = summary_table_throat_no_abs_prevl[['variable', 'count']]
summary_table_throat_abs_prevl = summary_table_throat_abs_prevl[['variable', 'count']]

In [668]:
#merge baseline dfs
dfs_tab = [summary_table_lrti_no_abs_incdt, summary_table_lrti_abs_incdt, summary_table_lrti_no_abs_prevl, summary_table_lrti_abs_prevl, 
        summary_table_urti_combined_no_abs_incdt, summary_table_urti_combined_abs_incdt, summary_table_urti_combined_no_abs_prevl, summary_table_urti_combined_abs_prevl,
        summary_table_uti_no_abs_incdt, summary_table_uti_abs_incdt, summary_table_uti_no_abs_prevl, summary_table_uti_abs_prevl,
        summary_table_pneumonia_no_abs_incdt, summary_table_pneumonia_abs_incdt, summary_table_pneumonia_no_abs_prevl, summary_table_pneumonia_abs_prevl,
        summary_table_sinusitis_no_abs_incdt, summary_table_sinusitis_abs_incdt, summary_table_sinusitis_no_abs_prevl, summary_table_sinusitis_abs_prevl,
        summary_table_otmedia_no_abs_incdt, summary_table_otmedia_abs_incdt, summary_table_otmedia_no_abs_prevl, summary_table_otmedia_abs_prevl,
        summary_table_ot_externa_no_abs_incdt, summary_table_ot_externa_abs_incdt, summary_table_ot_externa_no_abs_prevl, summary_table_ot_externa_abs_prevl,
        summary_table_urti_no_abs_incdt, summary_table_urti_abs_incdt, summary_table_urti_no_abs_prevl, summary_table_urti_abs_prevl,
        summary_table_cough_no_abs_incdt, summary_table_cough_abs_incdt, summary_table_cough_no_abs_prevl, summary_table_cough_abs_prevl,
        summary_table_cough_cold_no_abs_incdt, summary_table_cough_cold_abs_incdt, summary_table_cough_cold_no_abs_prevl, summary_table_cough_cold_abs_prevl,
        summary_table_throat_no_abs_incdt, summary_table_throat_abs_incdt, summary_table_throat_no_abs_prevl, summary_table_throat_abs_prevl
        ]

df_final_tab = ft.reduce(lambda left, right: pd.merge(left, right, on='variable', how='outer'), dfs_tab)

df_final_tab.columns = ['variable', 'lrti_no_abs_incdt', 'lrti_abs_incdt', 'lrti_no_abs_prevl', 'lrti_abs_prevl', 
        'urti_combined_no_abs_incdt', 'urti_combined_abs_incdt', 'urti_combined_no_abs_prevl', 'urti_combined_abs_prevl',
        'uti_no_abs_incdt', 'uti_abs_incdt', 'uti_no_abs_prevl', 'uti_abs_prevl',
        'pneumonia_no_abs_incdt', 'pneumonia_abs_incdt', 'pneumonia_no_abs_prevl', 'pneumonia_abs_prevl',
        'sinusitis_no_abs_incdt', 'sinusitis_abs_incdt', 'sinusitis_no_abs_prevl', 'sinusitis_abs_prevl',
        'otmedia_no_abs_incdt', 'otmedia_abs_incdt', 'otmedia_no_abs_prevl', 'otmedia_abs_prevl',
        'ot_externa_no_abs_incdt', 'ot_externa_abs_incdt', 'ot_externa_no_abs_prevl', 'ot_externa_abs_prevl',
        'urti_no_abs_incdt', 'urti_abs_incdt', 'urti_no_abs_prevl', 'urti_abs_prevl',
        'cough_no_abs_incdt', 'cough_abs_incdt', 'cough_no_abs_prevl', 'cough_abs_prevl',
        'cough_cold_no_abs_incdt', 'cough_cold_abs_incdt', 'cough_cold_no_abs_prevl', 'cough_cold_abs_prevl',
        'throat_no_abs_incdt', 'throat_abs_incdt', 'throat_no_abs_prevl', 'throat_abs_prevl'
        ]

#custom sort baseline dfs
df_final_tab['variable'] = pd.Categorical(df_final_tab['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                    'sex_female', 'sex_male',
                                                                    'CCI_cat_very_low', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_high', 'CCI_cat_very_high', 
                                                                    'flu_vaccine_no', 'flu_vaccine_yes',
                                                                    'bmi_cat_underweight', 'bmi_cat_healthy_weight', 'bmi_cat_overweight', 'bmi_cat_obese', 'bmi_cat_unknown',
                                                                    'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                    'imd_very_affluent', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_very_unaffluent', 'imd_unknown',
                                                                    'ethnicity_non-white', 'ethnicity_unknown', 'ethnicity_white',
                                                                    'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown',
                                                                    'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                    'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
                                                                    'antibacterial_brit'])
df_final_tab = df_final_tab.sort_values("variable").reset_index(drop=True)
# df_final_tab

In [669]:
#select relevant columns of each df
summary_table_hosped_lrti_no_abs_incdt = summary_table_hosped_lrti_no_abs_incdt[['variable', 'count']]
summary_table_hosped_lrti_abs_incdt = summary_table_hosped_lrti_abs_incdt[['variable', 'count']]
summary_table_hosped_lrti_no_abs_prevl = summary_table_hosped_lrti_no_abs_prevl[['variable', 'count']]
summary_table_hosped_lrti_abs_prevl = summary_table_hosped_lrti_abs_prevl[['variable', 'count']]
summary_table_hosped_urti_combined_no_abs_incdt = summary_table_hosped_urti_combined_no_abs_incdt[['variable', 'count']]
summary_table_hosped_urti_combined_abs_incdt = summary_table_hosped_urti_combined_abs_incdt[['variable', 'count']]
summary_table_hosped_urti_combined_no_abs_prevl = summary_table_hosped_urti_combined_no_abs_prevl[['variable', 'count']]
summary_table_hosped_urti_combined_abs_prevl = summary_table_hosped_urti_combined_abs_prevl[['variable', 'count']]
summary_table_hosped_uti_no_abs_incdt = summary_table_hosped_uti_no_abs_incdt[['variable', 'count']]
summary_table_hosped_uti_abs_incdt = summary_table_hosped_uti_abs_incdt[['variable', 'count']]
summary_table_hosped_uti_no_abs_prevl = summary_table_hosped_uti_no_abs_prevl[['variable', 'count']]
summary_table_hosped_uti_abs_prevl = summary_table_hosped_uti_abs_prevl[['variable', 'count']]
summary_table_hosped_pneumonia_no_abs_incdt = summary_table_hosped_pneumonia_no_abs_incdt[['variable', 'count']]
summary_table_hosped_pneumonia_abs_incdt = summary_table_hosped_pneumonia_abs_incdt[['variable', 'count']]
summary_table_hosped_pneumonia_no_abs_prevl = summary_table_hosped_pneumonia_no_abs_prevl[['variable', 'count']]
summary_table_hosped_pneumonia_abs_prevl = summary_table_hosped_pneumonia_abs_prevl[['variable', 'count']]
summary_table_hosped_sinusitis_no_abs_incdt = summary_table_hosped_sinusitis_no_abs_incdt[['variable', 'count']]
summary_table_hosped_sinusitis_abs_incdt = summary_table_hosped_sinusitis_abs_incdt[['variable', 'count']]
summary_table_hosped_sinusitis_no_abs_prevl = summary_table_hosped_sinusitis_no_abs_prevl[['variable', 'count']]
summary_table_hosped_sinusitis_abs_prevl = summary_table_hosped_sinusitis_abs_prevl[['variable', 'count']]
summary_table_hosped_otmedia_no_abs_incdt = summary_table_hosped_otmedia_no_abs_incdt[['variable', 'count']]
summary_table_hosped_otmedia_abs_incdt = summary_table_hosped_otmedia_abs_incdt[['variable', 'count']]
summary_table_hosped_otmedia_no_abs_prevl = summary_table_hosped_otmedia_no_abs_prevl[['variable', 'count']]
summary_table_hosped_otmedia_abs_prevl = summary_table_hosped_otmedia_abs_prevl[['variable', 'count']]
summary_table_hosped_ot_externa_no_abs_incdt = summary_table_hosped_ot_externa_no_abs_incdt[['variable', 'count']]
summary_table_hosped_ot_externa_abs_incdt = summary_table_hosped_ot_externa_abs_incdt[['variable', 'count']]
summary_table_hosped_ot_externa_no_abs_prevl = summary_table_hosped_ot_externa_no_abs_prevl[['variable', 'count']]
summary_table_hosped_ot_externa_abs_prevl = summary_table_hosped_ot_externa_abs_prevl[['variable', 'count']]
summary_table_hosped_urti_no_abs_incdt = summary_table_hosped_urti_no_abs_incdt[['variable', 'count']]
summary_table_hosped_urti_abs_incdt = summary_table_hosped_urti_abs_incdt[['variable', 'count']]
summary_table_hosped_urti_no_abs_prevl = summary_table_hosped_urti_no_abs_prevl[['variable', 'count']]
summary_table_hosped_urti_abs_prevl = summary_table_hosped_urti_abs_prevl[['variable', 'count']]
summary_table_hosped_cough_no_abs_incdt = summary_table_hosped_cough_no_abs_incdt[['variable', 'count']]
summary_table_hosped_cough_abs_incdt = summary_table_hosped_cough_abs_incdt[['variable', 'count']]
summary_table_hosped_cough_no_abs_prevl = summary_table_hosped_cough_no_abs_prevl[['variable', 'count']]
summary_table_hosped_cough_abs_prevl = summary_table_hosped_cough_abs_prevl[['variable', 'count']]
summary_table_hosped_cough_cold_no_abs_incdt = summary_table_hosped_cough_cold_no_abs_incdt[['variable', 'count']]
summary_table_hosped_cough_cold_abs_incdt = summary_table_hosped_cough_cold_abs_incdt[['variable', 'count']]
summary_table_hosped_cough_cold_no_abs_prevl = summary_table_hosped_cough_cold_no_abs_prevl[['variable', 'count']]
summary_table_hosped_cough_cold_abs_prevl = summary_table_hosped_cough_cold_abs_prevl[['variable', 'count']]
summary_table_hosped_throat_no_abs_incdt = summary_table_hosped_throat_no_abs_incdt[['variable', 'count']]
summary_table_hosped_throat_abs_incdt = summary_table_hosped_throat_abs_incdt[['variable', 'count']]
summary_table_hosped_throat_no_abs_prevl = summary_table_hosped_throat_no_abs_prevl[['variable', 'count']]
summary_table_hosped_throat_abs_prevl = summary_table_hosped_throat_abs_prevl[['variable', 'count']]

In [670]:
#merge hosped dfs
dfs_tab_hosped = [summary_table_hosped_lrti_no_abs_incdt, summary_table_hosped_lrti_abs_incdt, summary_table_hosped_lrti_no_abs_prevl, summary_table_hosped_lrti_abs_prevl, 
        summary_table_hosped_urti_combined_no_abs_incdt, summary_table_hosped_urti_combined_abs_incdt, summary_table_hosped_urti_combined_no_abs_prevl, summary_table_hosped_urti_combined_abs_prevl,
        summary_table_hosped_uti_no_abs_incdt, summary_table_hosped_uti_abs_incdt, summary_table_hosped_uti_no_abs_prevl, summary_table_hosped_uti_abs_prevl,
        summary_table_hosped_pneumonia_no_abs_incdt, summary_table_hosped_pneumonia_abs_incdt, summary_table_hosped_pneumonia_no_abs_prevl, summary_table_hosped_pneumonia_abs_prevl,
        summary_table_hosped_sinusitis_no_abs_incdt, summary_table_hosped_sinusitis_abs_incdt, summary_table_hosped_sinusitis_no_abs_prevl, summary_table_hosped_sinusitis_abs_prevl,
        summary_table_hosped_otmedia_no_abs_incdt, summary_table_hosped_otmedia_abs_incdt, summary_table_hosped_otmedia_no_abs_prevl, summary_table_hosped_otmedia_abs_prevl,
        summary_table_hosped_ot_externa_no_abs_incdt, summary_table_hosped_ot_externa_abs_incdt, summary_table_hosped_ot_externa_no_abs_prevl, summary_table_hosped_ot_externa_abs_prevl,
        summary_table_hosped_urti_no_abs_incdt, summary_table_hosped_urti_abs_incdt, summary_table_hosped_urti_no_abs_prevl, summary_table_hosped_urti_abs_prevl,
        summary_table_hosped_cough_no_abs_incdt, summary_table_hosped_cough_abs_incdt, summary_table_hosped_cough_no_abs_prevl, summary_table_hosped_cough_abs_prevl,
        summary_table_hosped_cough_cold_no_abs_incdt, summary_table_hosped_cough_cold_abs_incdt, summary_table_hosped_cough_cold_no_abs_prevl, summary_table_hosped_cough_cold_abs_prevl,
        summary_table_hosped_throat_no_abs_incdt, summary_table_hosped_throat_abs_incdt, summary_table_hosped_throat_no_abs_prevl, summary_table_hosped_throat_abs_prevl
        ]

df_final_tab_hosped = ft.reduce(lambda left, right: pd.merge(left, right, on='variable', how='outer'), dfs_tab_hosped)

df_final_tab_hosped.columns = ['variable', 'lrti_no_abs_incdt', 'lrti_abs_incdt', 'lrti_no_abs_prevl', 'lrti_abs_prevl', 
        'urti_combined_no_abs_incdt', 'urti_combined_abs_incdt', 'urti_combined_no_abs_prevl', 'urti_combined_abs_prevl',
        'uti_no_abs_incdt', 'uti_abs_incdt', 'uti_no_abs_prevl', 'uti_abs_prevl',
        'pneumonia_no_abs_incdt', 'pneumonia_abs_incdt', 'pneumonia_no_abs_prevl', 'pneumonia_abs_prevl',
        'sinusitis_no_abs_incdt', 'sinusitis_abs_incdt', 'sinusitis_no_abs_prevl', 'sinusitis_abs_prevl',
        'otmedia_no_abs_incdt', 'otmedia_abs_incdt', 'otmedia_no_abs_prevl', 'otmedia_abs_prevl',
        'ot_externa_no_abs_incdt', 'ot_externa_abs_incdt', 'ot_externa_no_abs_prevl', 'ot_externa_abs_prevl',
        'urti_no_abs_incdt', 'urti_abs_incdt', 'urti_no_abs_prevl', 'urti_abs_prevl',
        'cough_no_abs_incdt', 'cough_abs_incdt', 'cough_no_abs_prevl', 'cough_abs_prevl',
        'cough_cold_no_abs_incdt', 'cough_cold_abs_incdt', 'cough_cold_no_abs_prevl', 'cough_cold_abs_prevl',
        'throat_no_abs_incdt', 'throat_abs_incdt', 'throat_no_abs_prevl', 'throat_abs_prevl'
        ]

#custom sort factors of variables column 
df_final_tab_hosped['variable'] = pd.Categorical(df_final_tab_hosped['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                    'sex_female', 'sex_male',
                                                                    'CCI_cat_very_low', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_high', 'CCI_cat_very_high', 
                                                                    'flu_vaccine_no', 'flu_vaccine_yes',
                                                                    'bmi_cat_underweight', 'bmi_cat_healthy_weight', 'bmi_cat_overweight', 'bmi_cat_obese', 'bmi_cat_unknown',
                                                                    'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                    'imd_very_affluent', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_very_unaffluent', 'imd_unknown',
                                                                    'ethnicity_non-white', 'ethnicity_unknown', 'ethnicity_white',
                                                                    'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown',
                                                                    'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                    'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic', 
                                                                    'antibacterial_brit'])
df_final_tab_hosped = df_final_tab_hosped.sort_values("variable").reset_index(drop=True)
# df_final_tab_hosped

## 1- process data to make a table of counts and rates, and save it

In [671]:
data_hosped = data_process(df_final_tab, df_final_tab_hosped, 'lrti_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'pneumonia_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'pneumonia_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'pneumonia_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'pneumonia_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_abs_prevl')

data_hosped

,variable,lrti_no_abs_incdt,lrti_abs_incdt,lrti_no_abs_prevl,lrti_abs_prevl,urti_combined_no_abs_incdt,urti_combined_abs_incdt,urti_combined_no_abs_prevl,urti_combined_abs_prevl,uti_no_abs_incdt,...,cough_no_abs_prevl,cough_abs_prevl,cough_cold_no_abs_incdt,cough_cold_abs_incdt,cough_cold_no_abs_prevl,cough_cold_abs_prevl,throat_no_abs_incdt,throat_abs_incdt,throat_no_abs_prevl,throat_abs_prevl
0,age_cat_15_24,-,10 (4.4),15 (6.6),SN,600 (7.4),70 (7.8),80 (8.9),10 (9.7),145 (7.0),...,25 (11.0),SN,155 (7.4),20 (8.8),10 (4.1),-,170 (8.1),20 (8.7),20 (8.9),-
1,age_cat_25_34,160 (7.8),20 (8.9),10 (4.4),SN,595 (7.3),75 (8.2),65 (7.2),10 (9.9),160 (7.8),...,15 (6.5),SN,165 (7.9),25 (11.3),20 (8.9),SN,175 (8.5),20 (8.5),15 (6.6),SN
2,age_cat_35_44,145 (7.2),20 (9.0),25 (11.0),SN,575 (7.1),60 (6.6),60 (6.7),10 (9.5),170 (8.3),...,15 (6.7),SN,155 (7.4),20 (8.9),15 (6.3),SN,145 (7.0),15 (6.6),20 (8.7),SN
3,age_cat_45_54,130 (6.4),20 (9.2),20 (8.8),SN,640 (7.9),65 (7.0),60 (6.6),SN,165 (8.0),...,15 (6.6),SN,160 (7.8),20 (8.8),20 (8.8),SN,165 (8.0),20 (8.6),20 (8.6),-
4,age_cat_55_64,155 (7.7),20 (8.5),10 (4.6),-,580 (7.2),60 (6.7),60 (6.8),SN,175 (8.4),...,20 (9.0),SN,155 (7.6),20 (8.9),15 (6.4),SN,120 (5.7),15 (6.3),15 (6.6),SN
5,age_cat_65_74,150 (7.3),20 (8.9),20 (9.1),SN,610 (7.5),55 (6.1),60 (6.5),10 (9.6),160 (7.8),...,20 (8.4),SN,155 (7.5),25 (10.4),15 (6.6),SN,160 (7.8),15 (6.5),15 (6.4),SN
6,age_cat_75_more,135 (6.6),20 (8.8),15 (6.6),SN,600 (7.3),70 (7.5),65 (7.1),SN,165 (8.0),...,15 (6.7),SN,160 (7.6),15 (6.3),20 (8.7),SN,145 (6.9),20 (8.7),10 (4.2),SN
7,sex_female,525 (7.2),80 (9.9),60 (7.4),10 (11.2),"2,125 (7.3)",230 (7.1),235 (7.3),30 (8.3),590 (8.0),...,65 (7.9),10 (10.6),565 (7.6),65 (7.9),55 (6.6),SN,555 (7.5),65 (7.9),55 (6.7),SN
8,sex_male,490 (7.0),55 (7.1),60 (7.8),SN,"2,080 (7.5)",230 (7.4),220 (7.1),25 (7.2),550 (7.8),...,55 (7.1),10 (11.2),540 (7.6),65 (8.4),60 (7.7),SN,525 (7.3),65 (8.2),60 (7.6),SN
9,CCI_cat_very_low,200 (7.5),30 (10.5),30 (10.2),SN,745 (7.1),75 (6.3),75 (6.4),15 (11.3),215 (8.1),...,15 (5.2),SN,180 (6.7),20 (6.7),25 (8.5),SN,190 (7.0),25 (8.3),15 (5.1),SN


In [672]:
data_hosped[['variable', 'pneumonia_no_abs_incdt', 'pneumonia_abs_incdt', 'pneumonia_no_abs_prevl', 'pneumonia_abs_prevl']]

,variable,pneumonia_no_abs_incdt,pneumonia_abs_incdt,pneumonia_no_abs_prevl,pneumonia_abs_prevl
0,age_cat_15_24,160 (7.8),20 (8.5),20 (8.6),SN
1,age_cat_25_34,150 (7.3),10 (4.3),20 (8.9),SN
2,age_cat_35_44,150 (7.2),10 (4.3),20 (8.8),SN
3,age_cat_45_54,135 (6.6),15 (6.7),20 (8.8),SN
4,age_cat_55_64,155 (7.5),10 (4.2),15 (6.4),SN
5,age_cat_65_74,155 (7.4),20 (8.1),25 (10.7),SN
6,age_cat_75_more,130 (6.3),25 (11.0),15 (6.6),SN
7,sex_female,520 (7.0),55 (6.5),70 (8.5),SN
8,sex_male,515 (7.3),60 (7.6),65 (8.2),10 (12.1)
9,CCI_cat_very_low,205 (7.5),20 (6.5),30 (10.0),SN
